# Shuffling
- 그룹핑시 데이터를 한 노드에서 다른 노드로 옮길 때 발생, 성능을 (많이) 저하시킨다.
- 여러 네트워크 연산을 일으키기 때문에 네트워크 코스트가 크다.

### Shuffle을 일으킬 수 있는 작업들
- Join, leftOuterJoin, rightOuterJoin
- GroupByKey
- ReduceByKey
- CombineByKey
- Distinct
- Intersection
- Repartition
- Coalesce

### Shuffling은 언제 일어날까?
- 결과로 나오는 RDD가 원본 RDD의 다른 요소를 참조하거나 , 다른 RDD를 참조할 때

### Partitioner를 이용한 성능최적화 
- GroupByKeys + Reduce를 하게되면 => 성능을 매우 떨어짐


### Shuffle을 최소화 하려면
- 미리 파티션을 만들어 두고 캐싱 후 reduceByKey 실행
- 미리 파티션을 만들어 두고 캐싱 후 Join 실행
- 둘다 파티션과 캐싱을 조합해서 최대한 로컬 환경에서 연산이 실행되도록 하는 방식

***셔플을 최소화해서 10배의 성능 향상이 가능하다***

# 2. 예제

In [1]:
from operator import add
from pyspark import SparkConf, SparkContext

import warnings 
warnings.simplefilter(action='ignore')

conf = SparkConf().setMaster('local').setAppName('category-review-average')
sc = SparkContext(conf=conf)

In [2]:
# reduceByKey
# 임의의 텍스트 데이터
text_data = ["Spark is a fast and general-purpose cluster-computing system",
             "It provides high-level APIs in Java, Scala, Python and R",
             "Spark is built on the Hadoop Distributed File System"]

# 데이터를 RDD로 변환
textRDD = sc.parallelize(text_data)

# 단어 빈도 계산
word_frequency = (textRDD
    .flatMap(lambda line: line.split())            # 각 줄을 단어로 분할
    .map(lambda word: (word.lower(), 1))           # 각 단어를 소문자로 변환하고 (word, 1) 쌍을 생성
    .reduceByKey(lambda a, b: a + b))              # 같은 단어의 값을 합산

# 결과 출력
for word, count in word_frequency.collect():
    print(f"{word}: {count}")

spark: 2
is: 2
a: 1
fast: 1
and: 2
general-purpose: 1
cluster-computing: 1
system: 2
it: 1
provides: 1
high-level: 1
apis: 1
in: 1
java,: 1
scala,: 1
python: 1
r: 1
built: 1
on: 1
the: 1
hadoop: 1
distributed: 1
file: 1


In [4]:
# 임의의 텍스트 데이터
text_data = ["Spark is a fast and general-purpose cluster-computing system",
             "It provides high-level APIs in Java, Scala, Python and R",
             "Spark is built on the Hadoop Distributed File System"]

# 데이터를 RDD로 변환
textRDD = sc.parallelize(text_data)

# 단어 빈도 계산
word_frequency = (textRDD
    .flatMap(lambda line: line.split())              # 각 줄을 단어로 분할
    .map(lambda word: (word.lower(), 1))             # 각 단어를 소문자로 변환하고 (word, 1) 쌍을 생성
    .groupByKey()                                    # 같은 단어를 그룹화
    .map(lambda wc: (wc[0], sum(wc[1]))))            # 그룹화된 단어의 값을 합산

# 결과 출력
for word, count in word_frequency.collect():
    print(f"{word}: {count}")

spark: 2
is: 2
a: 1
fast: 1
and: 2
general-purpose: 1
cluster-computing: 1
system: 2
it: 1
provides: 1
high-level: 1
apis: 1
in: 1
java,: 1
scala,: 1
python: 1
r: 1
built: 1
on: 1
the: 1
hadoop: 1
distributed: 1
file: 1


# Partition은 어떻게 결정될까?

### 1) 데이터가 어느 노드 /파티션에 들어가는지는 어떻게 결정될까?

### 2) 파티션의 목적
- 데이터를 최대한 균일하게 퍼트리고 쿼리가 같이 되는 데이터를 최대한 옆에 두어 검색 성능을 향상시키는 것
- 파티션은 PairedRDD일 때만 의미가 있음, 일반 RDD는 처음부터 끝까지 scanning을 해야함(의미없음)
- 해시로 파티셔닝했을 때 어떤 데이터를 찾고 싶다 했을 때 Key를 통해 바로 찾을 수 있음

### 3) Partition의 특징
- RDD는 쪼개져서 여러 파티션에 저장됨
- 하나의 파티션은 하나의 노드 (서버)에 저장됨
- 하나의 노드는 여러개의 파티션을 가질 수 있음
- 파티션의 크기와 배치는 자유롭게 설정 가능하며 성능에 큰 영향을 미침
- Key-Value RDD를 사용할때만 의미가 있다.

***스파크의 파티셔닝 == 일반 프로그래밍에서 자료구조를 선택하는 것**

### 4) Partition의 종류
- Hash Partitioning
- Range Partitioning

##### (1) Hash Partitioning 
- 데이터를 여러 파티션에 균일하게 분배하는 방식
    - 자료 구조 중 Dictionary와 유사하다
    - key가 여러가지가 있을 때 key를 hash function을 통해 hash 값으로 반환한다.
    - 랜덤하게 나온 hash 값을 Key로 만들고 다시 data를 저장한다.
        - 데이터가 랜덤하게 분포
- [극단적인 예] 2개의 파티션이 있는 상황에서:
    - 짝수의 Key만 있는 데이터셋에 Hash 함수가 (x%2)라면?
    - 한쪽 파티션만 사용하게 될 것
    
    Partition 1: [2,4,6,8,10, ...]
    Partition 2: []

    데이터가 Skewed(쏠림) 됐음. 즉 적절한 hash 함수를 쓰는게 필요함
    
##### (2) Range Partitioning     
- 순서가 있는 정렬된 파티셔닝
    - 키의 순서에 따라
    - 키의 집합의 순서에 따라 
    
서비스의 쿼리 패턴이 날짜 위주면 일별 Range Partition 고려

### 5) Memory & Disk Partition

##### (1) 디스크에서 파티션하기
- PartitionBy()
- 사용자가 지정한 파티션을 가지는 RDD를 생성하는 함수
    - partitionBy()
    - 파티션을 만든 후엔 Persist()를 하지 않으면:
        - 다음 연산에 불릴때마다 반복하게 된다! (셔플링이 반복적으로 일어난다).


In [10]:
pairs = sc.parallelize([1,2,3,4,2,4,1]).map(lambda x: (x,x)) # 숫자를 입려받아 RDD를 생성. 
print(pairs.collect())

#Glom은 partition까지 보여주는 transformation 어떤 형상인지 알 수 있다.
# Hash Partitioning을 적용하며 파티션 수는 2다. 
# glom()은 RDD를 파티션별로 그룹화
print(pairs.partitionBy(2).glom().collect()) 

# 파티션을 hash 함수를 새로 지정해주고 싶다? => 짝수/홀수
print(pairs.partitionBy(2, lambda x: x%2).glom().collect())

[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]
[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]
[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]


##### (2) 메모리에서 파티션하기
- Repartition()
- coalesce()

##### Repartition과 Coalesce 둘다 파티션의 갯수를 조절하는데 사용
- 둘다 shuffling을 동반하여 매우 비싼 작업
- Repartition: 파티션의 크기를 줄이거나 늘리는데 사용됨
- Coalesce: 파티션의 크기를 줄이는데 사용됨

##### 아래의 함수들은 연산중에 새로운 파티션을 만들 수 있다.
- Join(leftOuterJoin, rightOuterJoin) 
- groupByKey
- reduceByKey
- foldByKey
- partitionBy
- Sort
- mapValues(Parent)
- flatMapValues(Parent)
- filter(Parent)
- 등등

- Map vs MapValues
- flatMap vs FlatMapValues
- map과 flatMap은 Key의 변형이 가능하기 때문